# Grid Search y Hyperparameters

Una técnica muy común que empleamos los científicos de datos es probar varios modelos con diferentes parámetros para no tener que estar adivinando.

## Hiperparámetros

Un hiperparámetro es un parámetro que se establece en un modelo dem achine learning. En otras palabras, son los argumentos que mandamos al constructor de la clase del modelo.

~~~python
DecisionTreeRegressor(max_leaf_nodes=50, random_state=0)
~~~

En este caso, `max_leaf_nodes` es un hiperparámetro.

Los hiperparámetros controlan el comportamiento del algoritmo de entrenamiento y la estructura del modelo. A diferencia de los parámetros del modelo, que se ajustan durante el entrenamiento (por ejemplo, los pesos en una regresión lineal), los hiperparámetros deben definirse antes de comenzar el entrenamiento.

### Ejemplos de Hiperparámetros

#### RandomForestRegressor 
- n_estimators: El número de árboles en el bosque.
- max_features: El número de características a considerar al buscar la mejor división.

#### GradientBoostingRegressor
- n_estimators: El número de etapas de incremento a ejecutar.
- learning_rate: Reduce la contribución de cada árbol a esta tasa. Hay un equilibrio entre learning_rate y n_estimators.

#### DecisionTreeRegressor
- max_depth: La profundidad máxima del árbol.
- min_samples_split: El número mínimo de muestras requerido para dividir un nodo interno.

#### SVR (Regresor de Máquinas de Vectores de Soporte):
- kernel: Especifica el tipo de núcleo a usar en el algoritmo (por ejemplo, linear, rbf).
- C: Parámetro de regularización. La fuerza de la regularización es inversamente proporcional a C.


---

### GridSearch

GridSearch (búsqueda en cuadrícula) es una técnica utilizada para encontrar la mejor combinación de hiperparámetros para un modelo. Ajustar correctamente estos hiperparámetros puede mejorar significativamente el rendimiento del modelo.

#### Proceso de GridSearch
1. Definición de Parámetros: Se especifica un conjunto de hiperparámetros y sus posibles valores. Esto se organiza en una cuadrícula (grid) utilizando un diccionario de Python.

2. Entrenamiento y validación: Para cada combinación de hiperparámetros en la cuadrícula, se entrena el modelo y se evalúa su rendimiento utilizando validación cruzada (cross-validation). Esto implica dividir los datos en varias partes, entrenar el modelo en algunas partes y evaluar en las restantes, repitiendo este proceso varias veces.

3. Selección del Mejor Modelo: Se selecciona la combinación de hiperparámetros que produce el mejor rendimiento según una métrica de evaluación específica, como el error absoluto medio (Mean Absolute Error, MAE) o la precisión (accuracy).

---

## Implementación

Importaremos pandas como siempre, varios modelos de Scikit Learn y algunas funciones de utilidad.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
import numpy as np

Ahora preparamos nuestros datos como ya lo sabemos hacer

In [ ]:

# Cargar datos
ruta = './data/melb_data.csv'
melbourne_data = pd.read_csv(ruta) 

# Quitar nulos
melbourne_data_filtrada = melbourne_data.dropna(axis=0)

# Elegir target (crear y)
y = melbourne_data_filtrada.Price

# Elegir features
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude']

# Crear X
X = melbourne_data_filtrada[melbourne_features]

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

A continuación, creamos un diccionario que contendrá los modelos que queremos utilizar en nuestro GridSearch

In [ ]:
# Definir los modelos y los parámetros para la búsqueda en grid
models = {
    'RandomForest': RandomForestRegressor(random_state=1),
    'GradientBoosting': GradientBoostingRegressor(random_state=1),
    'LinearRegression': LinearRegression(),
    'DecisionTree': DecisionTreeRegressor(random_state=1),
    'SVR': SVR()
}

Definimos los hiperparámetros para cada modelo también con un diccionario.

In [ ]:
# Definir los parámetros para cada modelo
params = {
    'RandomForest': {
        'n_estimators': [50, 100, 200]
    },
    'GradientBoosting': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    'LinearRegression': {},
    'DecisionTree': {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    },
    'SVR': {
        'kernel': ['linear', 'rbf'],
        'C': [0.1, 1, 10]
    }
}

Ejecutamos el GridSearch

In [ ]:
# Realizar la búsqueda en grid para cada modelo y almacenar los resultados
mejores_parametros = {}
mejor_score = {}

for model_name in models:
    print(f"Ajustando modelo: {model_name}")
    grid_search = GridSearchCV(models[model_name], params[model_name], cv=5, scoring='neg_mean_absolute_error')
    grid_search.fit(train_X, train_y)
    mejores_parametros[model_name] = grid_search.best_estimator_
    mejor_score[model_name] = -grid_search.best_score_
    print(f"Mejores parámetros para {model_name}: {grid_search.best_params_}")
    print(f"Mejor score de {model_name}: {mejor_score[model_name]}")
    print("=============================================\n")


Calcular MAE

In [ ]:
# Evaluar los mejores modelos en el conjunto de validación
maes = {}
for model_name in mejores_parametros:
    preds = mejores_parametros[model_name].predict(val_X)
    mae = mean_absolute_error(val_y, preds)
    maes[model_name] = mae
    print(f"{model_name} MAE: {mae}")

In [ ]:
print(f"Mejor modelo: {min(maes, key=maes.get)} con MAE: {np.min(list(maes.values()))}")